## Assignment 9 Consumer

The following notebook should help you with debugging and testing assignment 9. It creates a `KafkaConsumer` the subscribes to the `LastnameFirstname-simple`, `LastnameFirstname-windowed`, and `LastnameFirstname-joined` topics and prints any messages.

In [1]:
import json
import pandas as pd

from kafka import KafkaConsumer

### Configuration Parameters 

> **TODO:** Change the configuration prameters to the appropriate values for your setup.

In [2]:

config = dict(
    bootstrap_servers=['kafka.kafka.svc.cluster.local:9092'],
    first_name='Kurt',
    last_name='Stoneburner'
)

config['client_id'] = '{}{}'.format(
    config['last_name'], 
    config['first_name']
)
config['topic_prefix'] = '{}{}'.format(
    config['last_name'], 
    config['first_name']
)

config
config['simple_topic'] = '{}-simple'.format(config['topic_prefix'])
config['joined_topic'] = '{}-joined'.format(config['topic_prefix'])
config['windowed_topic'] = '{}-windowed'.format(config['topic_prefix'])

config

{'bootstrap_servers': ['kafka.kafka.svc.cluster.local:9092'],
 'first_name': 'Kurt',
 'last_name': 'Stoneburner',
 'client_id': 'StoneburnerKurt',
 'topic_prefix': 'StoneburnerKurt',
 'simple_topic': 'StoneburnerKurt-simple',
 'joined_topic': 'StoneburnerKurt-joined',
 'windowed_topic': 'StoneburnerKurt-windowed'}

Close the consumer, waiting indefinitely for any needed cleanup.

In [3]:
def create_kafka_consumer(topics, config=config):
    bootstrap_servers = config['bootstrap_servers']
    client_id = config['client_id']
    topic_prefix = config['topic_prefix']    
    topic_list = ['{}-{}'.format(topic_prefix, topic) for topic in topics]
    
    return KafkaConsumer(
        *topic_list,
        client_id=client_id,
        bootstrap_servers=bootstrap_servers,
        value_deserializer=lambda x: json.loads(x)
    )

consumer = create_kafka_consumer(['simple', 'windowed', 'joined'])
#consumer = create_kafka_consumer(['locations', 'accelerations'])

Gets a list of this consumer's current subscriptions

In [4]:
consumer.subscription()

{'StoneburnerKurt-joined',
 'StoneburnerKurt-simple',
 'StoneburnerKurt-windowed'}

The following function prints messages from the current consumer subscriptions. It will continue until manually stopped. 

In [5]:
from IPython.display import clear_output



def print_messages(consumer=consumer):
    try:
        for message in consumer:
                msg_metadata = 'Message metadata: {}:{}:{}'.format(
                    message.topic, message.partition, message.offset
                )

                if message.key is not None:
                    msg_key = message.key.decode('utf-8')
                else:
                    msg_key = ''
                msg_value = json.dumps(message.value, indent=2)
                msg_value = '\n'.join(['  {}'.format(value) for value in msg_value.split('\n')])
                clear_output(wait=True)
                print('Message metadata:')
                print('  Topic: {}'.format(message.topic))
                print('  Partition: {}'.format(message.partition))
                print('  Offset: {}'.format(message.offset))
                print('Message Key: {}'.format(msg_key))
                print('Message Value:')
                df = pd.read_json(json.loads(msg_value))
                #//*** Display the First 5 lines of the response converted to a DataFrame
                print(df.iloc[:5])
                
                print()
    except KeyboardInterrupt:
        print("STOPPING MESSAGE CONSUMER")



print_messages()

Message metadata:
  Topic: StoneburnerKurt-simple
  Partition: 0
  Offset: 5642
Message Key: a31fbcb6104a4ecea5d8c2254180505e
Message Value:
                                 id                           ride_id  \
0  d23ab293596c360dab6e8fcdb1160292  57ac74e28d873fb0b1c7d9998349f008   

                               uuid  timestamp      offset      course  \
0  8c6d4a328f8f4d1abd748eaa64db26b3    1506962  103.294091  138.164062   

    latitude  longitude       geohash  speed  accuracy  timelapse  \
0  40.612146 -73.925367  dr5rjc1nyhen      0        10      False   

                                   filename  
0  3e6093ea-22ec-47db-812c-47dec2b98d27.mov  

STOPPING MESSAGE CONSUMER


Close the consumer, waiting indefinitely for any needed cleanup.

In [6]:
consumer.close()